In [7]:
from zipfile import ZipFile
import os
from tensorflow import keras
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
import numpy as np


In [19]:
uri = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip"
zip_path = keras.utils.get_file(origin=uri, fname="jena_climate_2009_2016.csv.zip")
zip_file = ZipFile(zip_path)
cwd = os.getcwd()
csv_path = cwd+"/Data"
zip_file.extractall(path=csv_path)


In [22]:
df = pd.read_csv(csv_path+"/jena_climate_2009_2016.csv")
df.shape

(420551, 15)

In [25]:
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])
tscv = TimeSeriesSplit(n_splits=4,max_train_size=3)
print(tscv)

for train, test in tscv.split(X):
    print("%s %s" % (train, test))


TimeSeriesSplit(gap=0, max_train_size=3, n_splits=4, test_size=None)
[0 1] [2]
[0 1 2] [3]
[1 2 3] [4]
[2 3 4] [5]
